In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt

In [2]:
# Read in a dataframe for pandas. Use a path from your own directories. Files are too big to upload to git.
dfraw = pd.read_csv('bus_routes/46a_2012_raw.csv')

c:\users\minogud2\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Format of the unix timestamp needs to be altered to be converted to readible format. Conversion is from ms to seconds
dfraw['unix'] = dfraw['timestamp']//1000000
# Now convert to datetime format
dfraw['unix'] = pd.to_datetime(dfraw['unix'], unit='s')

In [4]:
# Make new column for day of the week. Eventually we may wish to query times according to day of week. 
dfraw['weekday'] = dfraw['unix'].dt.dayofweek

In [5]:
# Separate columns for date and column to allow easier iteration specific to time and not date
dfraw['date'] = [d.date() for d in dfraw['unix']]
dfraw['time'] = [d.time() for d in dfraw['unix']]

In [6]:
# # set the timestamp the index to make it easier to iterate over. 
dfraw = dfraw.set_index('unix')

In [7]:
# Group by only the weekday selected for travel. In this example 1 = Monday, so all mondays are selected.
gb = dfraw.groupby(['weekday']).get_group(1)

In [8]:
# retrieve only the travel times selected. In this example, we assume it's 8.00am. We selected 30+- around the time.
gb_active_buses = gb.between_time('07:59:30','08:00:30')

In [9]:
# create a list from all the buses that were historically in motion during the times selected. 
journeyList = set([])
for i in gb_active_buses.vehicle_journey_id:
    journeyList.add(i)

In [10]:
df_specific_buses = gb[gb['vehicle_journey_id'].isin(journeyList)]

In [11]:
# pick out only data that is at the stop to calculate distances between stops. 
df_specific_buses = df_specific_buses.ix[(df_specific_buses['at_stop'] == 1)]

In [12]:
# dataset reduced to 10,000 rows. Still statistically significant for our predictions. 
df_specific_buses.shape

(9468, 19)

In [13]:
df_specific_buses.head(5)

,timestamp,line_id,direction,journey_pattern_id,time_frame,vehicle_journey_id,operator,congestion,lat,lon,delay,block_id,vehicle_id,stop_id,at_stop,unique_id,weekday,date,time
unix,,,,,,,,,,,,,,,,,,,
2012-11-06 06:47:09,1352184429000000,46,0,046A0001,2012-11-06,7289,D2,0,-6.297616,53.351665,0.0,46007,36008,807,1,54556,1,2012-11-06,06:47:09
2012-11-06 06:47:48,1352184468000000,46,0,046A0001,2012-11-06,7289,D2,0,-6.297616,53.351665,0.0,46007,36008,807,1,55310,1,2012-11-06,06:47:48
2012-11-06 06:47:50,1352184470000000,46,0,046A0001,2012-11-06,7289,D2,0,-6.297616,53.351665,0.0,46007,36008,807,1,55600,1,2012-11-06,06:47:50
2012-11-06 06:48:08,1352184488000000,46,0,046A0001,2012-11-06,7289,D2,0,-6.297616,53.351665,0.0,46007,36008,807,1,56002,1,2012-11-06,06:48:08
2012-11-06 06:48:28,1352184508000000,46,0,046A0001,2012-11-06,7289,D2,0,-6.297616,53.351665,0.0,46007,36008,807,1,56392,1,2012-11-06,06:48:28


In [14]:
# Include only buses taking off from the time specified.
df_specific_buses = df_specific_buses.between_time('07:59:30', df_specific_buses.time.max())

In [15]:
# Now group by only the journey_id, stop_id and timestamp.This creates a pandas series.
df_specific_buses = df_specific_buses.groupby([df_specific_buses.index.values, 'vehicle_journey_id', 'stop_id']).size()

In [16]:
# reset index and convert into a dataframe from it's series.
df_specific_buses = df_specific_buses.to_frame().reset_index()

In [17]:
type(df_specific_buses)
# check type to make sure the alteration was made

pandas.core.frame.DataFrame

In [18]:
# sort the dataset according to journey and the timestamp(level_0)
df_specific_buses = df_specific_buses.sort_values(by=['vehicle_journey_id','level_0'])

In [19]:
# df_specific_buses = x[x.level_0 > '2012-11-06 07:59:30']

In [20]:
# Group the data then by vehicle and stop
gb_stops = df_specific_buses.groupby(['vehicle_journey_id', 'stop_id'])

In [21]:
# Reduce the repetition of at_stop=1 by selecting the first the bus enters the stop as the arrival time
gb_stops = gb_stops.agg(lambda x: x.iloc[0])

In [22]:
# reset the index to allow for sorting by vehicle and timestamp
gb_stops = gb_stops.reset_index(level=False)

In [24]:
# Sort by vehicle and timestamp.
gb_stops = gb_stops.sort_values(by=['vehicle_journey_id','level_0'])

In [25]:
# Calculate the difference between the timestamp from one stop to the next by vehicle_journey_id
gb_stops['difference'] = gb_stops.groupby(gb_stops.vehicle_journey_id).level_0.apply(lambda x: x - x.iloc[0])

In [26]:
gb_stops.index

Int64Index([2007, 2008, 2009, 2010,  435, 4571, 4636, 2017, 2021, 2022,
            ...
            2014, 4636, 2022, 4565, 2034, 2035, 2039,  809,  812,  814],
           dtype='int64', name='stop_id', length=1689)

In [28]:
# Print out the result. 
gb_stops.tail(50)

,vehicle_journey_id,level_0,0,difference
stop_id,,,,
2009,7431,2012-11-06 08:48:18,1,0 days 00:47:50
4571,7431,2012-11-06 08:52:55,1,0 days 00:52:27
2013,7431,2012-11-06 08:53:57,1,0 days 00:53:29
2021,7431,2012-11-06 09:03:19,1,0 days 01:02:51
2022,7431,2012-11-06 09:03:56,1,0 days 01:03:28
4565,7431,2012-11-06 09:06:36,1,0 days 01:06:08
2035,7431,2012-11-06 09:13:16,1,0 days 01:12:48
2039,7431,2012-11-06 09:22:01,1,0 days 01:21:33
2014,7431,2012-11-13 08:58:48,1,7 days 00:58:20
